Installing dependencies

In [1]:
%%script echo skipping
# %pip install binance-connector
%pip install tradingview-ta
%pip install pandas
%pip install python-dotenv
%pip install python-binance

skipping


Trying out TradingView API

In [2]:
%%script echo skipping
from tradingview_ta import TA_Handler, Interval, Exchange

ADA = TA_Handler(
    symbol="ADAUSDT",
    screener="crypto",
    exchange="BINANCE",
    interval=Interval.INTERVAL_1_DAY,
    # proxies={'http': 'http://example.com:8080'} # Uncomment to enable proxy (replace the URL).
)
print(ADA.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

skipping


In [3]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import requests
from binance.client import Client

In [4]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_PASSWORD']
ta_api = os.environ['taapi_key']


# client = Client(api_key, api_secret)
# print('Using Binance TestNet Server')
# print('')
# print(binance_client.get_account())
# client = Client(base_url='https://testnet.binance.vision')
# print(client.time())
# print(client.account())

# from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret, testnet=True)
# client.ping()
# client.get_server_time()
# info = client.get_symbol_info('ETHUSDT')
# print(info)

In [5]:
def get_data_frame(symbol):
    # starttime = '6 months ago UTC'
    starttime = '1 Jan, 2017'
    interval = '1h'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])

    return df

get_data_frame('ETHUSDT')

,date,open,high,low,close
0,1667394000000,1556.31000000,1557.38000000,1414.70000000,1553.45000000
1,1667397600000,1553.25000000,1572.20000000,1523.86000000,1560.21000000
2,1667401200000,1560.22000000,1561.57000000,1481.81000000,1558.54000000
3,1667404800000,1558.51000000,1574.85000000,1550.09000000,1571.23000000
4,1667408400000,1570.95000000,1577.10000000,1550.91000000,1558.57000000
...,...,...,...,...,...
116,1667811600000,1568.08000000,1575.22000000,1500.00000000,1564.37000000
117,1667815200000,1563.88000000,1584.32000000,1536.01000000,1578.53000000
118,1667818800000,1578.29000000,3421.00000000,1393.13000000,1579.96000000
119,1667822400000,1579.85000000,1584.98000000,1576.49000000,1584.57000000


In [6]:
def macd_logic(symbol):

    symbol_df = get_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['200_ema'] < symbol_df['close'])), 1, 0)

    return symbol_df

macd_logic('ETHUSDT')

,date,open,high,low,close,macd,signal,200_ema,Trigger
0,2022-11-02 13:00:00,1556.31,1557.38,1414.70,1553.45,0.000000,0.000000,1553.450000,0
1,2022-11-02 14:00:00,1553.25,1572.20,1523.86,1560.21,0.539259,0.107852,1553.517264,1
2,2022-11-02 15:00:00,1560.22,1561.57,1481.81,1558.54,0.822391,0.250760,1553.567241,1
3,2022-11-02 16:00:00,1558.51,1574.85,1550.09,1571.23,2.047154,0.610038,1553.742990,1
4,2022-11-02 17:00:00,1570.95,1577.10,1550.91,1558.57,1.973482,0.882727,1553.791020,1
...,...,...,...,...,...,...,...,...,...
116,2022-11-07 09:00:00,1568.08,1575.22,1500.00,1564.37,-14.376212,-11.763272,1585.201483,0
117,2022-11-07 10:00:00,1563.88,1584.32,1536.01,1578.53,-13.380135,-12.086644,1585.135100,0
118,2022-11-07 11:00:00,1578.29,3421.00,1393.13,1579.96,-12.333179,-12.135951,1585.083607,0
119,2022-11-07 12:00:00,1579.85,1584.98,1576.49,1584.57,-11.004615,-11.909684,1585.078496,0


# BACKTESTING

Backtesting completed on 6 month historical data on 5m time frame

In [7]:
def get_six_month_data_frame(symbol):
    # starttime = '6 months ago UTC'
    starttime = '1 Jan, 2017'
    interval = '1h'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])
    return df

six_month_eth_df_raw = get_six_month_data_frame('ETHUSDT')

In [45]:
# url1 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1660629600000'
# url2 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1664229600000'
# url3 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1667829600000'

def get_bars(symbol, limit = '1000', interval = '1h'):
    df = pd.DataFrame()
    root_url = 'https://api.binance.com/api/v1/klines'
    end_times = ['1660629600000', '1664229600000', '1667829600000']
    for time in end_times:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval + '&limit=' + limit + '&endTime=' + time
        data = json.loads(requests.get(url).text)
        for row in data:
            del row[5:]
        df = df.append(data)
    df.columns = ['date',
                'open', 'high', 'low', 'close']
    return df.reset_index()

get_bars('ETHUSDT')

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_48189/1687454806.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


,index,date,open,high,low,close
0,0,1657033200000,1093.38000000,1095.08000000,1076.34000000,1092.95000000
1,1,1657036800000,1092.95000000,1103.50000000,1088.28000000,1095.60000000
2,2,1657040400000,1095.60000000,1107.41000000,1094.33000000,1102.68000000
3,3,1657044000000,1102.69000000,1128.00000000,1099.44000000,1127.51000000
4,4,1657047600000,1127.50000000,1152.75000000,1123.82000000,1146.53000000
...,...,...,...,...,...,...
2995,995,1667815200000,1564.10000000,1584.56000000,1563.88000000,1578.54000000
2996,996,1667818800000,1578.53000000,1582.02000000,1575.66000000,1580.00000000
2997,997,1667822400000,1579.99000000,1585.02000000,1576.48000000,1584.51000000
2998,998,1667826000000,1584.50000000,1589.73000000,1575.31000000,1579.30000000


In [8]:
six_month_eth_df_raw

,date,open,high,low,close
0,1667394000000,1556.31000000,1557.38000000,1414.70000000,1553.45000000
1,1667397600000,1553.25000000,1572.20000000,1523.86000000,1560.21000000
2,1667401200000,1560.22000000,1561.57000000,1481.81000000,1558.54000000
3,1667404800000,1558.51000000,1574.85000000,1550.09000000,1571.23000000
4,1667408400000,1570.95000000,1577.10000000,1550.91000000,1558.57000000
...,...,...,...,...,...
116,1667811600000,1568.08000000,1575.22000000,1500.00000000,1564.37000000
117,1667815200000,1563.88000000,1584.32000000,1536.01000000,1578.53000000
118,1667818800000,1578.29000000,3421.00000000,1393.13000000,1579.96000000
119,1667822400000,1579.85000000,1584.98000000,1576.49000000,1584.57000000


In [9]:
def test_macd_logic(symbol):

    symbol_df = get_six_month_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['close'] > symbol_df['200_ema']) & (symbol_df['macd'] < 0) & (symbol_df['signal'] > symbol_df['macd'].shift())), 1, 0)
    
    tr_list = [(symbol_df['high'] - symbol_df['low']), (symbol_df['low'] - symbol_df['close'].shift()), (symbol_df['high'] - symbol_df['close'].shift())]

    symbol_df['TR1'] = symbol_df['high'] - symbol_df['low']
    symbol_df['prev close'] = symbol_df['close'].shift()
    symbol_df['TR2'] = symbol_df['low'] - symbol_df['close'].shift()
    symbol_df['TR3'] = symbol_df['high'] - symbol_df['close'].shift()
    # symbol_df['TR LIST'] = symbol_df.apply(lambda row : [(row['high'] - row['low']), (row['low'] - row['close'].shift()), (row['high'] - row['close'].shift())])
    # symbol_df['TR LIST'] = symbol_df.apply(symbol_df)

    symbol_df = symbol_df.iloc[200:]

    triggered_df = symbol_df.loc[symbol_df['Trigger'] == 1]

    return symbol_df

test_df = test_macd_logic('ETHUSDT')

In [10]:
test_df

,date,open,high,low,close,macd,signal,200_ema,Trigger,TR1,prev close,TR2,TR3


In [11]:
atr_indicator = "atr"
atr_endpoint = f"https://api.taapi.io/{atr_indicator}"

live_price_indicator = "price"
live_price_endpoint = f"https://api.taapi.io/{atr_indicator}"

stop_loss = 0
tp = 0
entry = 0

parameters = {
    'secret': ta_api,
    'exchange': 'binance',
    'symbol': 'ETH/USDT',
    'interval': '1H'
    } 

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

atr_test = 16.74326363683623

for index, row in test_df.iterrows():
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True

            stop_loss = row['close'] - atr_test
            tp = row['close'] + atr_test
            entry = row['close']
            
    
    else:
        if row['close'] <= stop_loss:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print('')

        elif row['close'] >= tp and tp1_hit == False:
            # in_trade = False
            close = row['close']
            tp1_hit = True
            stop_loss = entry
            win += 1
            profit += ((tp - entry)/2)
            print(f'TP1: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')
            tp = (entry + (2*atr_test))
            
        elif row['close'] >= tp and tp1_hit == True:
            close = row['close']
            in_trade = False
            tp1_hit = False
            profit += ((tp - entry)/2)
            print(f'TP2: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')


        
print(f'wins: {win}, profit: ${profit}')
print(f'losses: {loss}, losses: ${losses}')
print(f'P/L: {profit - losses}')

wins: 0, profit: $0
losses: 0, losses: $0
P/L: 0


In [12]:
eth_raw_df_2 = get_six_month_data_frame('ETHUSDT')
eth_raw_df_2

,date,open,high,low,close
0,1667394000000,1556.31000000,1557.38000000,1414.70000000,1553.45000000
1,1667397600000,1553.25000000,1572.20000000,1523.86000000,1560.21000000
2,1667401200000,1560.22000000,1561.57000000,1481.81000000,1558.54000000
3,1667404800000,1558.51000000,1574.85000000,1550.09000000,1571.23000000
4,1667408400000,1570.95000000,1577.10000000,1550.91000000,1558.57000000
...,...,...,...,...,...
116,1667811600000,1568.08000000,1575.22000000,1500.00000000,1564.37000000
117,1667815200000,1563.88000000,1584.32000000,1536.01000000,1578.53000000
118,1667818800000,1578.29000000,3421.00000000,1393.13000000,1579.96000000
119,1667822400000,1579.85000000,1584.98000000,1576.49000000,1584.57000000


In [13]:
import requests        # for making http requests to binance
import json            # for parsing what binance sends back to us
import pandas as pd    # for storing and manipulating the data we get back
import numpy as np     # numerical python, i usually need this somewhere 
                       # and so i import by habit nowadays

import matplotlib.pyplot as plt # for charts and such
    
import datetime as dt  # for dealing with times

def get_bars(symbol, interval = '1h'):
   root_url = 'https://api.binance.com/api/v1/klines'
   url = root_url + '?symbol=' + symbol + '&interval=' + interval
   data = json.loads(requests.get(url).text)
   for row in data:
        del row[5:]
   df = pd.DataFrame(data)
   df.columns = ['date',
                 'open', 'high', 'low', 'close']
   return df

steemeth = get_bars('STEEMETH')
ethusdt = get_bars('ETHUSDT').astype('float')
# steemusdt = steemeth['c'].astype('float') * ethusdt['c'].astype('float')
# ethusdt['c'].astype('float').plot(figsize=(16,9))

# steemusdt.plot(figsize=(16,9))
# steemeth

In [14]:
def test_logic_2(raw_df):
    raw_df['open'] = pd.to_numeric(raw_df['open'])
    raw_df['high'] = pd.to_numeric(raw_df['high'])
    raw_df['low'] = pd.to_numeric(raw_df['low'])
    raw_df['close'] = pd.to_numeric(raw_df['close'])
    raw_df['date'] = pd.to_datetime(raw_df['date'], unit = 'ms')


    short_ema = raw_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = raw_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = raw_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    raw_df['macd'] = macd
    raw_df['signal'] = signal
    raw_df['200_ema'] = ema_200

    raw_df['TR1'] = abs(raw_df['high'] - raw_df['low'])
    raw_df['TR2'] = abs(raw_df['low'] - raw_df['close'].shift())
    raw_df['TR3'] = abs(raw_df['high'] - raw_df['close'].shift())

    raw_df['ATR'] = raw_df.apply(lambda x: row_check(x.TR1, x.TR2, x.TR3), axis=1)

    raw_df = raw_df.iloc[200:]

    raw_df = raw_df.drop(['TR1', 'TR2', 'TR3'], axis=1)

    raw_df['Trigger'] = np.where(((raw_df['macd'] > raw_df['signal']) & (raw_df['close'] > raw_df['200_ema']) & (raw_df['macd'] < 0) & (raw_df['signal'] > raw_df['macd'].shift())), 1, 0)

    return raw_df

def row_check(TR1, TR2, TR3):
    if TR1 >= TR2 and TR1 >= TR3:
        ATR = TR1
    elif TR2 >= TR1 and TR2 >= TR3:
        ATR = TR2
    else:
        ATR = TR3
    
    return ATR

In [15]:
# tinkered_df = test_logic_2(eth_raw_df_2)
tinkered_df = test_logic_2(ethusdt)

In [24]:
1504771200000
print(pd.to_datetime(1504771200000, unit = 'ms'))

2017-09-07 08:00:00


In [16]:
tinkered_df

,date,open,high,low,close,macd,signal,200_ema,ATR,Trigger
200,2022-10-26 02:00:00,1480.37,1491.98,1476.49,1485.72,34.760592,31.476458,1338.258596,15.49,0
201,2022-10-26 03:00:00,1485.71,1499.21,1479.89,1487.03,34.526564,32.086479,1339.738909,19.32,0
202,2022-10-26 04:00:00,1487.02,1490.02,1476.83,1479.95,33.384957,32.346175,1341.134044,13.19,0
203,2022-10-26 05:00:00,1479.95,1485.25,1477.48,1484.34,32.460280,32.368996,1342.558979,7.77,0
204,2022-10-26 06:00:00,1484.35,1501.16,1482.49,1497.73,32.434048,32.382006,1344.102969,18.67,0
...,...,...,...,...,...,...,...,...,...,...
495,2022-11-07 09:00:00,1568.08,1575.58,1564.10,1564.10,-14.410638,-11.808442,1580.406464,11.48,0
496,2022-11-07 10:00:00,1564.10,1584.56,1563.88,1578.54,-13.407928,-12.128339,1580.387892,20.68,0
497,2022-11-07 11:00:00,1578.53,1582.02,1575.66,1580.00,-12.353065,-12.173284,1580.384032,6.36,0
498,2022-11-07 12:00:00,1579.99,1585.02,1576.48,1584.51,-11.026057,-11.943839,1580.425087,8.54,1


In [17]:
stop_loss = 0
tp = 0
entry = 0

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

for index, row in tinkered_df.iterrows():
    atr = 0
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True
            atr = row['ATR']
            stop_loss = row['close'] - atr
            tp = row['close'] + atr
            entry = row['close']
            
    
    else:
        if row['close'] <= stop_loss:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print('')

        elif row['close'] >= tp and tp1_hit == False:
            # in_trade = False
            close = row['close']
            tp1_hit = True
            stop_loss = entry
            win += 1
            profit += ((tp - entry)/2)
            print(f'TP1: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')
            tp = (entry + (2*atr))
            
        elif row['close'] >= tp and tp1_hit == True:
            close = row['close']
            in_trade = False
            tp1_hit = False
            profit += ((tp - entry)/2)
            print(f'TP2: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')


        
print(f'wins: {win}, profit: ${profit}')
print(f'losses: {loss}, losses: ${losses}')
print(f'P/L: {profit - losses}')

TP1: 1580.8000000000002, CLOSE: 1589.52, ENTRY: 1540.18
Profit: 20.31000000000006

TP2: 1540.18, CLOSE: 1581.04, ENTRY: 1540.18
Profit: 20.31000000000006

TP1: 1597.8700000000001, CLOSE: 1620.76, ENTRY: 1589.16
Profit: 24.665000000000077

TP2: 1589.16, CLOSE: 1623.06, ENTRY: 1589.16
Profit: 24.665000000000077

TP1: 1595.87, CLOSE: 1598.35, ENTRY: 1581.18
Profit: 32.00999999999999

TP2: 1581.18, CLOSE: 1586.81, ENTRY: 1581.18
Profit: 32.00999999999999

STOP LOSS: 1567.1899999999998, CLOSE: 1564.76, ENTRY: 1587.54
Losses: 20.350000000000136

TP1: 1551.5, CLOSE: 1553.34, ENTRY: 1543.59
Profit: 35.96500000000003

TP2: 1543.59, CLOSE: 1551.76, ENTRY: 1543.59
Profit: 35.96500000000003

TP1: 1582.7, CLOSE: 1587.43, ENTRY: 1551.79
Profit: 51.42000000000007

TP2: 1551.79, CLOSE: 1579.33, ENTRY: 1551.79
Profit: 51.42000000000007

STOP LOSS: 1624.08, CLOSE: 1620.35, ENTRY: 1628.83
Losses: 25.100000000000136

STOP LOSS: 1612.89, CLOSE: 1610.59, ENTRY: 1626.42
Losses: 38.63000000000011

wins: 5, pr

In [18]:
candles = client.get_klines(symbol='BNBBTC', interval=Client.KLINE_INTERVAL_1HOUR)

In [19]:
pd.DataFrame(candles)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1667394000000,0.01568200,0.01568200,0.00390000,0.01564800,624.75000000,1667397599999,9.59646010,1307,48.66000000,0.76193091,0
1,1667397600000,0.01564700,0.01570200,0.00390000,0.01563300,500.19000000,1667401199999,7.62861701,1103,106.01000000,1.65555072,0
2,1667401200000,0.01562900,0.01564400,0.01548200,0.01560400,69.92000000,1667404799999,1.09175447,200,39.25000000,0.61322331,0
3,1667404800000,0.01560900,0.01587500,0.01456000,0.01587300,301.12000000,1667408399999,4.71991370,997,200.33000000,3.14176859,0
4,1667408400000,0.01587500,0.01617100,0.00390000,0.01570300,572.66000000,1667411999999,8.96649552,1835,239.30000000,3.78647093,0
...,...,...,...,...,...,...,...,...,...,...,...,...
116,1667811600000,0.01600300,0.01600700,0.01594400,0.01596800,79.57000000,1667815199999,1.27144123,247,75.97000000,1.21395661,0
117,1667815200000,0.01596400,0.01605300,0.01594400,0.01602500,109.56000000,1667818799999,1.75455352,341,104.88000000,1.67967263,0
118,1667818800000,0.01602600,0.01606700,0.01600100,0.01602600,77.68000000,1667822399999,1.24560214,244,75.29000000,1.20728584,0
119,1667822400000,0.01602200,0.01602500,0.01592200,0.01598300,137.35000000,1667825999999,2.19522095,441,133.27000000,2.13002785,0


In [20]:
import requests

response = requests.get('https://cnft.tools/api/whitelist/autowl')

data = response.json()
pioneer_count = 0
wl_count = 0

for test in data:
    print('Pioneer')
    if test['role'] == '1018984763291340840':
        pioneer_count += 1
        print(test['role'])
        print(test['addr'])
        print('')

for test in data:
    print('WL')
    if test['role'] == '1019005107800637490':
        wl_count += 1
        print(test['role'])
        print(test['addr'])
        print('')

print(pioneer_count)
print(wl_count)

Pioneer
1018984763291340840
addr1qxrhwpe3gw75pp8ux7p7tshvac34dr0yhtrpj9g2jjgahccs98m6apttgmutjlekx08zk4uy6sk24v5apkvv9xsnznzqulfa49

Pioneer
1018984763291340840
addr1q8prqclry8mluma8sejt67rp6juryv9pmzep7ar4afp7jalguhf4g7srq59xeklh8xwatjd29qu4s7aam8p06ylatgvspx7cjn

Pioneer
1018984763291340840
addr1q9y2w3u7t0xpzmrue8846rd8sy9h5geu8h443rumkfxhlt545aest3dwktdnk76u6npcufryax4xd9ley3mhe5mxruls5zhvup

Pioneer
1018984763291340840
addr1qxcy9xwkdj2gsyg4xpzxqt6vssrr2tuvz964pfzlu3gfeus09tqs5rppqwtj25xjn5t6k9jmg5hrp8y0zsw064wu0kcs9f3hjl

Pioneer
1018984763291340840
addr1qxq4e7804g980q00ddpdkrhgh9el3h4uewczs2aef25ut764mya972xe80ufcreq42zhv6f2ghfkpa0jacsyzhvfs84ssa0x3j

Pioneer
1018984763291340840
addr1q9dk7r0kr5r8wfnm0vc6xzqe0nwj7fydn6pa34577edysrnk5elmd8nutfq6vl07epvj56my84rz455ctqnz6yv9m97qqfvlpz

Pioneer
1018984763291340840
addr1qx48v6xdgqyead9mym8w2hje7f4sk640jfdlhlamg25ranakevsa0fjalzt8udlw5ntcwfwz9jmx92g4wsnl7uxdwnvs5yv5g2

Pioneer
1018984763291340840
addr1qxyjemk5z2l9lwrs2xhvkq2slykltqa0krn2

In [21]:
# indicator = "atr"
# endpoint = f"https://api.taapi.io/{indicator}"
# parameters = {
#     'secret': ta_api,
#     'exchange': 'binance',
#     'symbol': 'ETH/USDT',
#     'interval': '15m'
#     } 
# response = requests.get(url = endpoint, params = parameters)
result = response.json() 
print(result['value'])

TypeError: list indices must be integers or slices, not str

In [27]:
import requests        # for making http requests to binance
import json            # for parsing what binance sends back to us
import pandas as pd    # for storing and manipulating the data we get back
import numpy as np     # numerical python, i usually need this somewhere 
                       # and so i import by habit nowadays

import matplotlib.pyplot as plt # for charts and such
    
import datetime as dt  # for dealing with times

def get_bars(symbol, limit = '1000', interval = '15m'):
   root_url = 'https://api.binance.com/api/v1/klines'
   url = root_url + '?symbol=' + symbol + '&interval=' + interval + '&limit=' + limit
   data = json.loads(requests.get(url).text)
   for row in data:
        del row[5:]
   df = pd.DataFrame(data)
   df.columns = ['date',
                 'open', 'high', 'low', 'close']
   return df

steemeth = get_bars('STEEMETH')
ethusdt = get_bars('ETHUSDT').astype('float')
# steemusdt = steemeth['c'].astype('float') * ethusdt['c'].astype('float')
# ethusdt['c'].astype('float').plot(figsize=(16,9))

# steemusdt.plot(figsize=(16,9))
ethusdt
# steemeth

,date,open,high,low,close
0,1.666930e+12,1508.56,1510.06,1507.47,1508.55
1,1.666931e+12,1508.54,1510.00,1507.28,1509.39
2,1.666932e+12,1509.40,1510.36,1506.78,1508.24
3,1.666933e+12,1508.23,1508.25,1501.80,1503.54
4,1.666934e+12,1503.54,1510.30,1501.71,1507.45
...,...,...,...,...,...
995,1.667826e+12,1584.50,1589.73,1581.28,1583.29
996,1.667827e+12,1583.28,1584.58,1575.31,1578.31
997,1.667828e+12,1578.31,1581.24,1576.73,1578.56
998,1.667829e+12,1578.57,1581.93,1576.14,1579.30


In [34]:
1664229600000 + 3600000

1664233200000

In [33]:
1660633200000 - 3600000

1660629600000

In [44]:
# url1 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1660629600000'
# url2 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1664229600000'
# url3 = 'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1667829600000'

def get_bars(symbol, limit = '1000', interval = '1h'):
    df = pd.DataFrame()
    root_url = 'https://api.binance.com/api/v1/klines'
    end_times = ['1660629600000', '1664229600000', '1667829600000']
    for time in end_times:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval + '&limit=' + limit + '&endTime=' + time
        data = json.loads(requests.get(url).text)
        for row in data:
            del row[5:]
        df = df.append(data)
    df.columns = ['date',
                'open', 'high', 'low', 'close']
    return df.reset_index()

get_bars('ETHUSDT')

/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_48189/1687454806.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


,index,date,open,high,low,close
0,0,1657033200000,1093.38000000,1095.08000000,1076.34000000,1092.95000000
1,1,1657036800000,1092.95000000,1103.50000000,1088.28000000,1095.60000000
2,2,1657040400000,1095.60000000,1107.41000000,1094.33000000,1102.68000000
3,3,1657044000000,1102.69000000,1128.00000000,1099.44000000,1127.51000000
4,4,1657047600000,1127.50000000,1152.75000000,1123.82000000,1146.53000000
...,...,...,...,...,...,...
2995,995,1667815200000,1564.10000000,1584.56000000,1563.88000000,1578.54000000
2996,996,1667818800000,1578.53000000,1582.02000000,1575.66000000,1580.00000000
2997,997,1667822400000,1579.99000000,1585.02000000,1576.48000000,1584.51000000
2998,998,1667826000000,1584.50000000,1589.73000000,1575.31000000,1579.30000000
